# Import libraries and configure settings

In [0]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from tqdm import tqdm_notebook as tqdm
from google.colab import drive
import os

# Connect to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Mount Google Drive
drive.mount('/content/drive')

# Prepare for importing modules
!pip install import-ipynb
import import_ipynb
# Install PyDrive wrapper and import libraries
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Prepare dataset

In [0]:
# Import prepare_dataset module and variables
prepare_dataset = drive.CreateFile({'id':'15l7vAAf7L4sVRUuRbLwIedrN3vhwQUfb'})
prepare_dataset.GetContentFile('prepare_dataset.ipynb')
from prepare_dataset import testloader, subset_list, transform_aug_list, num_classes, num_subset, aug_total

importing Jupyter notebook from prepare_dataset.ipynb
Files already downloaded and verified
Files already downloaded and verified


# Initialize non-IBE

In [0]:
# Import init_non_IBE module and variables
init_non_IBE = drive.CreateFile({'id':'1Fg9_nuYr2dHV8oSQrTDZJsnf2vltS-DA'})
init_non_IBE.GetContentFile('init_non_IBE.ipynb')
from init_non_IBE import net, criterion, optimizer, lr_scheduler

net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

importing Jupyter notebook from init_non_IBE.ipynb


# Initialize IBE

In [0]:
# Import init_IBE module and variables
init_IBE = drive.CreateFile({'id':'1k4JdlYX0ANxVEFDBxxp_pcqBzc2p_u2F'})
init_IBE.GetContentFile('init_IBE.ipynb')
from init_IBE import net_IBE, attention, criterion_IBE, optimizer_IBE, lr_scheduler_IBE

net_IBE.to(device)
if device == 'cuda':
    net_IBE = torch.nn.DataParallel(net_IBE)
    cudnn.benchmark = True

importing Jupyter notebook from init_IBE.ipynb


# Initialize variables

In [0]:
start_epoch = 0     # start from epoch 0 or last checkpoint epoch
end_epoch = 30
resumed = False
resume_aug_set_idx = 0
resume_set_idx = 0

# Load checkpoint

In [0]:
resume_model = 'IBE'           ### ENTER MODEL ###
resume_aug_set_idx = 2      ### ENTER AUG SET INDEX ###
resume_set_idx = 3          ### ENTER SET INDEX ###

# Load network state
checkpoint = torch.load('checkpoint_{}/aug_{}/ckpt_{}.pth'.format(resume_model, resume_aug_set_idx, resume_set_idx))
if resume_model == 'IBE':
    crt_net = net_IBE
    crt_criterion = criterion_IBE
    crt_optimizer = optimizer_IBE
    crt_lr_scheduler = lr_scheduler_IBE
else:
    crt_net = net
    crt_criterion = criterion
    crt_optimizer = optimizer
    crt_lr_scheduler = lr_scheduler
crt_net.load_state_dict(checkpoint['net'])
crt_optimizer.load_state_dict(checkpoint['optimizer'])
crt_lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
start_epoch = checkpoint['epoch']
resumed = True
print("Starting augmented set: ", resume_aug_set_idx)
print("Starting set: ", resume_set_idx)
print("Starting epoch: ", start_epoch)

Starting augmented set:  2
Starting set:  3
Starting epoch:  28


# Train

In [0]:
if resumed:
    set_idx = resume_set_idx
    aug_idx = resume_aug_set_idx
    set_list_ = subset_list[resume_set_idx:]

### Specify model to train ('IBE' | 'non-IBE') ###
crt_model = 'non-IBE'

def reset_model():
    init_state = torch.load('initial_state_{}.pth'.format(crt_model))
    if crt_model == 'IBE':
        crt_net = net_IBE
        crt_criterion = criterion_IBE
        crt_optimizer = optimizer_IBE
        crt_lr_scheduler = lr_scheduler_IBE
    else:
        crt_net = net
        crt_criterion = criterion
        crt_optimizer = optimizer
        crt_lr_scheduler = lr_scheduler
    
    crt_net.load_state_dict(init_state['net'])
    crt_optimizer.load_state_dict(init_state['optimizer'])
    crt_lr_scheduler.load_state_dict(init_state['lr_scheduler'])
    return crt_net, crt_criterion, crt_optimizer, crt_lr_scheduler

In [0]:
for a_ in range(resume_aug_set_idx, aug_total):
    if not resumed:
        set_list_ = list.copy(subset_list)
        aug_idx = a_

    crt_transform = transform_aug_list[aug_idx]
    for setloader in set_list_:
        if not resumed:
            # Reset model to initial state before running next set of data
            crt_net, crt_criterion, crt_optimizer, crt_lr_scheduler = reset_model()
            start_epoch = 0
            set_idx = subset_list.index(setloader)
        resumed = False

        for epoch in range(start_epoch, end_epoch):
            crt_net.train()
            epoch_loss = 0.0
            correct = 0
            total = 0
            train_pbar = tqdm(setloader)
            # batch size = 128, image-wise labels (10 classes)
            for i, (x, _label) in enumerate(train_pbar):

                # data augmentation
                to_PIL = transforms.ToPILImage()
                for k in range(len(x)):
                    img_PIL = to_PIL(x[k])
                    x[k] = crt_transform(img_PIL)

                x = x.cuda()
                _label = _label.cuda()  # _label size: [128], each element [0-9]

                if crt_model == 'IBE':
                    label = F.one_hot(_label, num_classes=10).float()   # label size: [128, 10], each element [0 or 1]
                    seg_out = crt_net(x)  # seg_out size: [128, 10, 32, 32]
                    attn = attention(seg_out)   # attn size: [128, 1, 32, 32], each element [0-1]
                    logit = torch.log(torch.exp(seg_out*0.5).mean((-2,-1)))*2   # logit size: [128, 10]
                else:
                    label = _label
                    logit = crt_net(x)

                crt_optimizer.zero_grad()
                loss = crt_criterion(logit, label)
                loss.backward()
                crt_optimizer.step()
                epoch_loss += loss.item()
                total += _label.size(0)
                _, predicted = logit.max(1)
                correct += predicted.eq(_label).sum().item()
                train_pbar.set_description('E={} A={} S={} Acc: {:.3f}%'.format(epoch, aug_idx, set_idx, 100.*correct/total))
            
            crt_lr_scheduler.step()

            # Save model
            state = {
                'net': crt_net.state_dict(),
                'optimizer': crt_optimizer.state_dict(),
                'lr_scheduler': crt_lr_scheduler.state_dict(),
                'epoch': epoch,
            }
            if not os.path.isdir('checkpoint_{}/aug_{}'.format(crt_model, aug_idx)):
                os.makedirs('checkpoint_{}/aug_{}'.format(crt_model, aug_idx))
            torch.save(state, 'checkpoint_{}/aug_{}/ckpt_{}.pth'.format(crt_model, aug_idx, set_idx))